In [1]:
#-----Base-----#
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#-----RDKit-----#
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem, PandasTools, Draw, Descriptors, MACCSkeys, rdFingerprintGenerator
from rdkit.ML.Descriptors import MoleculeDescriptors
from rdkit.Chem.MolStandardize import rdMolStandardize

In [3]:
class Representacao:

    def __init__(self, dataframe: pd.DataFrame) -> None:
        '''
            Input:
                - dataframe: pd.DataFrame contendo a representação SMILES
        '''

        self.dataframe = dataframe.copy()

    def fingerprint(
        self, 
        col_smiles: str, 
        fingerprint: str='morgan', 
        radius: int=2, 
        fpSize: int=2048, 
        use_count: bool=False
    ) -> pd.DataFrame:
        
        '''
            Input:
                - fingerprint: algoritmo de representação vetorial
                - col_smiles: nome da coluna contendo a representação canônica SMILES
                - col_fp: nome da coluna contendo a representação fingerprint do tipo UIntSparseIntVect ou ExplicitBitVect
        '''
        df = self.mol_to_frame(col_smiles=col_smiles)

        if fingerprint == 'maccs':
            df = self.fp_MACCS(col_frames='ROMol')
        else:
            df = self.fp_Morgan(col_frames='ROMol', radius=radius, fpSize=fpSize, use_count=use_count)

        df.dropna(ignore_index=True, inplace=True)
        df = self.bitVect_to_array('Fingerprint')

        return df
        
    def mol_to_frame(self, col_smiles: str) -> pd.DataFrame:
        '''
            Descrição:
                Gera a representação em frame da molécula, a partir da representação SMILES.
        
            Input:
                - col_smiles: nome da coluna contendo a representação canônica SMILES
        '''
    
        PandasTools.AddMoleculeColumnToFrame(frame=self.dataframe, smilesCol=col_smiles)
    
        return self.dataframe

    def bitVect_to_array(self, col_fp: str) -> pd.DataFrame:
        '''
            Descrição:
                Gera a representação em array a partir do fingerprint (UIntSparseIntVect ou ExplicitBitVect).
        
            Input:
                - col_fp: nome da coluna contendo a representação fingerprint do tipo UIntSparseIntVect ou ExplicitBitVect
        '''

        # Inicia uma lista vazia para receber os fingerprints em arrays
        fp_array = []
        
        # Define o fingerprint em array de cada registro
        for index in self.dataframe.index:
            try:
                fp = np.zeros((0, ), dtype=np.int8)
                DataStructs.ConvertToNumpyArray(self.dataframe[col_fp][index], fp)
            except Exception as e:
                fp = np.nan

            fp_array.append(fp)
    
        # Cria a coluna com o array fingerprint
        self.dataframe['Features'] = fp_array
    
        return self.dataframe
    
    def fp_MACCS(self, col_frames: str) -> pd.DataFrame:
        '''
            Descrição:
                Gera a representação fingerprint MACCS.
        
            Input:
                - col_frames: nome da coluna contendo a molécula em frame
        '''
    
        # Inicia uma lista vazia para receber a representação MACCS
        maccs_lista = []
        
        # Define a representação MACCS de cada registro
        for index in self.dataframe.index:
            try:
                maccs = MACCSkeys.GenMACCSKeys(self.dataframe[col_frames].iloc[index])
            except:
                maccs = np.nan

            maccs_lista.append(maccs)
    
        # Cria a coluna com MACCS fingerprint
        self.dataframe['Fingerprint'] = maccs_lista
    
        return self.dataframe
    
    def fp_Morgan(self, col_frames: str, radius: int=2, fpSize: int=2048, use_count: bool=False) -> pd.DataFrame:
        '''
            Descrição:
                Gera a representação fingerprint Morgan.
        
            Input:
                - col_frames: nome da coluna contendo a molécula em frame
                - radius: tamanho do raio para avaliação de moléculas vizinhas
                - fpSize: tamanho do fingerprint obtido
                - use_count: define se usa a função GetCountFingerprint() que retorna o número de ocorrências de cada subestrutura
        '''
    
        # Inicia uma lista vazia para receber a representação Morgan
        morgan_lista = []
        
        # Define a representação Morgan de cada registro
        for index in self.dataframe.index:
            morgan_gen = AllChem.GetMorganGenerator(radius=radius, fpSize=fpSize)
    
            try:
                # Se use_count = True, gera o fingerprint de contagem de subestruturas
                if use_count:
                    morgan = morgan_gen.GetCountFingerprint(self.dataframe[col_frames].iloc[index])

                # Se use_count = False, gera o fingerprint binário
                else:
                    morgan = morgan_gen.GetFingerprint(self.dataframe[col_frames].iloc[index])
                    
            except:
                morgan = np.nan
                
            morgan_lista.append(morgan)
    
        # Cria a coluna com Morgan fingerprint
        self.dataframe['Fingerprint'] = morgan_lista
    
        return self.dataframe